Copyright 2021-2023 @ Shenzhen Bay Laboratory & Peking University & Huawei Technologies Co., Ltd

This code is a part of MindSPONGE:
MindSpore Simulation Package tOwards Next Generation molecular modelling.

MindSPONGE is open-source software based on the AI-framework:
MindSpore (https://www.mindspore.cn/)

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.

You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.

See the License for the specific language governing permissions and
limitations under the License.

MindSPONGE advanced tutorial 01: Collective variables (CVs), metrics and analyse.

In [1]:
from mindspore import context
from sponge import Sponge
from sponge import ForceField
from sponge import set_global_units
from sponge import Protein
from sponge import UpdaterMD
from sponge.optimizer import SteepestDescent
from sponge.control import VelocityVerlet
from sponge.callback import WriteH5MD, RunInfo
from sponge.control import Langevin
from sponge.function import VelocityGenerator
from sponge.colvar import Torsion

In [2]:
context.set_context(mode=context.GRAPH_MODE, device_target="GPU")
set_global_units('nm', 'kj/mol')

In [3]:
system = Protein(pdb='alad.pdb')
potential = ForceField(system, 'AMBER.FF14SB')

In [4]:
phi = Torsion([4, 6, 8, 14])
psi = Torsion([6, 8, 14, 16])

In [5]:
min_opt = SteepestDescent(system.trainable_params(), 1e-7)
mini = Sponge(system, potential, min_opt, metrics={'phi': phi, 'psi': psi})

[MindSPONGE] Using fully connected neighbour list (not updated).


In [6]:
cv = mini.analyse()
for k, v in cv.items():
    print(k, v)


phi [[3.1415927]]
psi [[3.1415927]]


In [7]:
run_info = RunInfo(10)
mini.run(100, callbacks=[run_info])

[MindSPONGE] Started simulation at 2023-05-18 15:37:05
[MindSPONGE] Step: 0, E_pot: -55.75643, phi: -3.140886, psi: -3.1410766
[MindSPONGE] Step: 10, E_pot: -58.40555, phi: -3.1340268, psi: -3.1368413
[MindSPONGE] Step: 20, E_pot: -60.168476, phi: -3.1277595, psi: -3.1337605
[MindSPONGE] Step: 30, E_pot: -61.549706, phi: -3.1221528, psi: -3.131402
[MindSPONGE] Step: 40, E_pot: -62.696198, phi: -3.1171532, psi: -3.1295478
[MindSPONGE] Step: 50, E_pot: -63.674084, phi: -3.112676, psi: -3.1280725
[MindSPONGE] Step: 60, E_pot: -64.52254, phi: -3.1086385, psi: -3.1268952
[MindSPONGE] Step: 70, E_pot: -65.268196, phi: -3.1049676, psi: -3.1259575
[MindSPONGE] Step: 80, E_pot: -65.93031, phi: -3.1016028, psi: -3.1252148
[MindSPONGE] Step: 90, E_pot: -66.52346, phi: -3.0984938, psi: -3.1246324
[MindSPONGE] Finished simulation at 2023-05-18 15:37:08
[MindSPONGE] Simulation time: 3.51 seconds.
--------------------------------------------------------------------------------


In [8]:
cv = mini.analyse()
for k, v in cv.items():
    print(k, v)

phi [[-3.0958812]]
psi [[-3.1242208]]


In [9]:
vgen = VelocityGenerator(300)
velocity = vgen(system.shape, system.atom_mass)
opt = UpdaterMD(
    system,
    integrator=VelocityVerlet(system),
    thermostat=Langevin(system, 300),
    time_step=1e-3,
    velocity=velocity
)

In [10]:
md = Sponge(system, potential, opt, metrics={'phi': phi, 'psi': psi})

[MindSPONGE] Using fully connected neighbour list (not updated).


In [11]:
cb_h5md = WriteH5MD(system, 'tutorial_a01.h5md', save_freq=10)
md.run(1000, callbacks=[run_info, cb_h5md])

[MindSPONGE] Started simulation at 2023-05-18 15:37:13
[MindSPONGE] Step: 0, E_pot: -67.05887, E_kin: 83.896034, E_tot: 16.837166, Temperature: 320.3293, phi: -3.0950527, psi: -3.1256313
[MindSPONGE] Step: 10, E_pot: -21.73536, E_kin: 43.650623, E_tot: 21.915264, Temperature: 166.66551, phi: -3.0601666, psi: 3.1363442
[MindSPONGE] Step: 20, E_pot: -20.185848, E_kin: 64.18463, E_tot: 43.998783, Temperature: 245.06786, phi: -2.9961138, psi: 3.0761726
[MindSPONGE] Step: 30, E_pot: -30.493969, E_kin: 79.76107, E_tot: 49.2671, Temperature: 304.54135, phi: -2.9470093, psi: 2.9815526
[MindSPONGE] Step: 40, E_pot: -30.656712, E_kin: 66.40468, E_tot: 35.747967, Temperature: 253.54434, phi: -2.857146, psi: 2.9238849
[MindSPONGE] Step: 50, E_pot: -49.14535, E_kin: 92.3302, E_tot: 43.18485, Temperature: 352.5324, phi: -2.7445993, psi: 2.9843612
[MindSPONGE] Step: 60, E_pot: -28.700012, E_kin: 68.57472, E_tot: 39.87471, Temperature: 261.82996, phi: -2.7263236, psi: 3.0381215
[MindSPONGE] Step: 70, 

In [12]:
cv = md.analyse()
for k, v in cv.items():
    print(k, v)

phi [[-2.4289787]]
psi [[2.7230694]]
